In [2]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from src.db import DataBase
from src.utils_general import beeps
from src.utils_date import add_days
from src.utils_stocks import get_ls_sym
from src.utils_stocks import get_df_prices
try:
    dir_db = json.load(open('dir.txt'))['dir_db']
except:
    dir_db = ''
    assert dir_db, 'Please provide a directory to save the sqlite database, in the "dir_db" variable!'

def db_remove_dups_prices_m(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_m
         WHERE DATE(datetime)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_m
                WHERE DATE(datetime)>='{}'
                GROUP BY sym, datetime)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_prices_d(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_d
         WHERE DATE(date)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_d
                WHERE DATE(date)>='{}'
                GROUP BY sym, date)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_stocks(db):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM stocks
         WHERE ROWID NOT IN
               (SELECT MAX(ROWID)
                  FROM stocks
                 GROUP BY sym)
    '''
    db.execute(q)

ls_init_str = [
    #prices_m
    '''CREATE TABLE IF NOT EXISTS prices_m(
        sym TEXT
        ,datetime TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER
        ,is_reg_hours INTEGER)''',
    
    '''CREATE INDEX IF NOT EXISTS index_prices_m_all
        ON prices_m(sym, date(datetime), is_reg_hours)''',

    '''CREATE INDEX IF NOT EXISTS index_prices_m_date
        ON prices_m(date(datetime))''',

    #prices_d
    '''CREATE TABLE IF NOT EXISTS prices_d(
        sym TEXT
        ,date TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER)''',
   
    '''CREATE INDEX IF NOT EXISTS index_prices_d_date
        ON prices_d(sym, date(date))''',
    
    #stocks
    '''CREATE TABLE IF NOT EXISTS stocks(
        sym TEXT
        ,long_name TEXT
        ,sec TEXT
        ,ind TEXT
        ,quote_type TEXT
        ,fund_family TEXT
        ,summary TEXT
        ,timestamp TEXT)''',

    '''CREATE INDEX IF NOT EXISTS index_stocks
        ON stocks(sym, quote_type)''',
    
    #stocks_error
    '''CREATE TABLE IF NOT EXISTS stocks_error(
        sym TEXT)''',
    
    #proba
    '''CREATE TABLE IF NOT EXISTS proba(
        sym TEXT
        ,datetime TEXT
        ,my_index INTEGER
        ,proba REAL
        ,datetime_update TEXT)''',
]

db = DataBase(ls_init_str, dir_db)

# Update table: stocks

In [20]:
def get_df_info(sym):
    dt_info = yf.Ticker(sym).info
    dt_info['timestamp'] = datetime.datetime.now()
    dt_info['sector'] = dt_info.get('sector')
    dt_col = {
        'symbol':'sym',
        'longName':'long_name',
        'sector':'sec',
        'industry':'ind',
        'quoteType':'quote_type',
        'fundFamily':'fund_family',
        'longBusinessSummary':'summary',
        'timestamp':'timestamp',
    }
    dt_info = {key:dt_info.get(key) for key in dt_col}
    df = pd.DataFrame([dt_info])
    df = df.rename(columns=dt_col)
    return df

#get ls_sym
ls_sym = get_ls_sym()
q = '''
    --SELECT sym FROM STOCKS UNION ALL
    SELECT sym FROM stocks_error UNION ALL
    SELECT sym FROM stocks WHERE summary IS NOT NULL
'''
ls_sym_exclude = pd.read_sql(q, db.conn)['sym'].to_list()
ls_sym = [x for x in ls_sym if x not in ls_sym_exclude]
#extract and load
ls_df = []
for i, sym in enumerate(ls_sym):
    try:
        if (i+1)%50==0 and ls_df:
            df = pd.concat(ls_df)
            print('Saving...')
            df.to_sql('stocks', db.conn, if_exists='append', index=0)
            ls_df = []
        print(sym, i+1, len(ls_sym), end=' ')
        df_info = get_df_info(sym)
        ls_df.append(df_info)
        print('Done.')
    except Exception as e:
        print(f'Skipped... ({type(e).__name__}: {str(e)})')
        df = pd.DataFrame([{'sym':sym}])
        df.to_sql('stocks_error', db.conn, if_exists='append', index=0)
#load remaining
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('stocks', db.conn, if_exists='append', index=0)
#remove duplicates
db_remove_dups_stocks(db)
beeps()

Extracting symbols from URL 1/2...
Extracting symbols from URL 2/2...


['KTF',
 'PSN',
 'BCEI',
 'IEMG',
 'NWBI',
 'TNAV',
 'VBLT',
 'OPTN',
 'DFE',
 'BV',
 'SOHOB',
 'AGLE',
 'CWS',
 'SPTS',
 'IWO',
 'SV',
 'DRIP',
 'BSCT',
 'VLT',
 'NODK',
 'DRI',
 'CAAS',
 'BEAM',
 'FNI',
 'UNM',
 'RYF',
 'IJH',
 'CENT',
 'ICFI',
 'PZT',
 'JJE',
 'CBAT',
 'BLMN',
 'CND.U',
 'LEJU',
 'BRMK',
 'USDU',
 'CALA',
 'FCCY',
 'ADPT',
 'BTWN',
 'NYCB',
 'ANDE',
 'CW',
 'SPMD',
 'YSAC',
 'METC',
 'AVAL',
 'YRD',
 'NTRSO',
 'JD',
 'GKOS',
 'DFNV',
 'VRP',
 'PMAR',
 'FORTY',
 'DTEC',
 'CHIR',
 'DMAY',
 'ACAC',
 'SOHOO',
 'JPS',
 'ORI',
 'VRSK',
 'AGGY',
 'IWF',
 'GEN',
 'BTWNW',
 'SPSM',
 'USHG',
 'SHOO',
 'CNYA',
 'TSN',
 'SSD',
 'ALCO',
 'BSCE',
 'TADS',
 'CMRE',
 'SKOR',
 'ITIC',
 'JELD',
 'TC',
 'PBCTP',
 'APLS',
 'UNVR',
 'UFPT',
 'XHE',
 'ITA',
 'BFY',
 'ESPR',
 'JBGS',
 'CSX',
 'NSYS',
 'LDUR',
 'FSFG',
 'EINC',
 'RH',
 'GGG',
 'FISR',
 'SON',
 'KOP',
 'HPP',
 'DOGZ',
 'EVR',
 'NPN',
 'VCYT',
 'AGIO',
 'EXPR',
 'VEGN',
 'CRBN',
 'LEGN',
 'FTFT',
 'KBWR',
 'HZON',
 'FCPI',
 

# Update table: prices_m

In [3]:
def get_ls_df_prices_m(ls_sym, ls_date_str, ls_sym_exclude=[]):
    ls_df = []
    for i, sym in enumerate(ls_sym):
        try:
            if i%100==0:
                print(i+1, len(ls_sym))
            df = get_df_prices(sym, ls_date_str[0], ls_date_str[-1])
            len_unique_dates = len(df['datetime'].dt.date.unique())
            len_candles_per_day = df.shape[0]/len_unique_dates
            if len_unique_dates<len(ls_date_str):
                print('Incomplete data, skipping...', sym, len_unique_dates)
            elif len_candles_per_day<200:
                print('Not enough candles per day, skipping...', sym, len_candles_per_day)
            else:
                ls_df.append(df)
                ls_sym_exclude.append(sym)
        except Exception as e:
            print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
    return ls_df

# get max date present
q = '''
    SELECT DATE(MAX(datetime))
      FROM prices_m
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# get missing dates
df = yf.download('IBM', period='1y', interval='1d', progress=0).reset_index()
df['Date']= df['Date'].astype('str')
ls_date_str = df[df['Date']>max_date_str]['Date'].to_list()
assert ls_date_str, 'No dates to extract!'
print('Extracing prices in ', ls_date_str)
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract and load
ls_df = get_ls_df_prices_m(ls_sym, ls_date_str)
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('prices_m', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_m(db, ls_date_str[0])
    beeps()

Extracing prices in  ['2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04']
1 4253
Not enough candles per day, skipping... NYV 5.0
Not enough candles per day, skipping... HKIB 2.6
Incomplete data, skipping... GNRSU 4
Not enough candles per day, skipping... CCF 58.4
Not enough candles per day, skipping... CHI 168.6
Not enough candles per day, skipping... PHT 110.4
Not enough candles per day, skipping... SBT 55.4
Incomplete data, skipping... FISK 3

1 Failed download:
- GPAQ: No data found for this date range, symbol may be delisted
Error: GPAQ 'Datetime'
Incomplete data, skipping... LIVKU 3

1 Failed download:
- FMCI: No data found for this date range, symbol may be delisted
Error: FMCI 'Datetime'
Not enough candles per day, skipping... LXU 112.0
Not enough candles per day, skipping... GSV 172.8
Not enough candles per day, skipping... PLPC 19.2
Not enough candles per day, skipping... FBMS 77.8
Not enough candles per day, skipping... RLI 125.8
Not enough candles per day, 

Not enough candles per day, skipping... UBCP 6.6
Not enough candles per day, skipping... CLPR 84.8
Not enough candles per day, skipping... CYBE 148.2
Not enough candles per day, skipping... NSSC 69.4
Not enough candles per day, skipping... PIRS 192.8
Not enough candles per day, skipping... MYI 98.0
Not enough candles per day, skipping... ASLN 137.2

1 Failed download:
- CVTI: No data found for this date range, symbol may be delisted
Error: CVTI 'Datetime'
Not enough candles per day, skipping... ASPN 140.8
Not enough candles per day, skipping... NHA 15.6
Not enough candles per day, skipping... OFLX 17.0
Not enough candles per day, skipping... NYMTM 28.0
Not enough candles per day, skipping... KRUS 81.0
Not enough candles per day, skipping... DFP 72.2
Not enough candles per day, skipping... CRNX 131.2
Not enough candles per day, skipping... BME 67.4
Not enough candles per day, skipping... BBCP 156.4
Not enough candles per day, skipping... HOFT 64.0
Not enough candles per day, skipping...

Not enough candles per day, skipping... SCPH 69.4
601 4253
Not enough candles per day, skipping... BOCH 41.6
Not enough candles per day, skipping... CEA 44.4
Not enough candles per day, skipping... GROW 138.6
Not enough candles per day, skipping... INDB 103.6
Not enough candles per day, skipping... RA 173.0

1 Failed download:
- JFK: No data found for this date range, symbol may be delisted
Error: JFK 'Datetime'
Not enough candles per day, skipping... PSF 66.2
Not enough candles per day, skipping... PHI 84.2
Not enough candles per day, skipping... ANIK 94.6
Not enough candles per day, skipping... IIM 124.0
Not enough candles per day, skipping... TPVG 190.4
Not enough candles per day, skipping... NVMI 112.6
Not enough candles per day, skipping... OYST 108.2
Not enough candles per day, skipping... AKTX 106.8

1 Failed download:
- TZACU: No data found for this date range, symbol may be delisted
Error: TZACU 'Datetime'
Not enough candles per day, skipping... REPH 183.4
Not enough candles p

Not enough candles per day, skipping... NP 159.2
Not enough candles per day, skipping... TOTA 11.4
Not enough candles per day, skipping... BNY 48.6
Not enough candles per day, skipping... SCM 116.2
Not enough candles per day, skipping... NNI 70.6
Not enough candles per day, skipping... MDLY 63.2
Not enough candles per day, skipping... GTYH 69.4
Not enough candles per day, skipping... TAC 192.0
Not enough candles per day, skipping... PGP 54.2
Not enough candles per day, skipping... CPSS 30.8
Not enough candles per day, skipping... TUFN 150.4
Not enough candles per day, skipping... BUI 75.0

1 Failed download:
- ASFI: No data found for this date range, symbol may be delisted
Error: ASFI 'Datetime'
Not enough candles per day, skipping... HBIO 62.6
Not enough candles per day, skipping... MIND 79.6
Not enough candles per day, skipping... NCB 7.2
Not enough candles per day, skipping... ARC 97.2
Not enough candles per day, skipping... GPRK 135.8
Not enough candles per day, skipping... WRLD 62

Not enough candles per day, skipping... NBO 16.0

1 Failed download:
- PER: No data found for this date range, symbol may be delisted
Error: PER 'Datetime'
Not enough candles per day, skipping... BBU 84.2
Not enough candles per day, skipping... ASGN 188.0
Not enough candles per day, skipping... VSEC 27.4
Not enough candles per day, skipping... BIPC 194.6

1 Failed download:
- LOGM: No data found for this date range, symbol may be delisted
Error: LOGM 'Datetime'
Not enough candles per day, skipping... CRHM 70.4
Not enough candles per day, skipping... CHN 20.8
Not enough candles per day, skipping... OSPN 199.6
Not enough candles per day, skipping... LOV 66.0
Not enough candles per day, skipping... VLT 71.0
Not enough candles per day, skipping... ULH 77.4
Not enough candles per day, skipping... MITO 112.6

1 Failed download:
- RTIX: No data found for this date range, symbol may be delisted
Error: RTIX 'Datetime'
Not enough candles per day, skipping... CIICU 92.4
Incomplete data, skipping.

...


1 Failed download:
- NFIN: No data found for this date range, symbol may be delisted
Error: NFIN 'Datetime'
Not enough candles per day, skipping... TPL 31.8
Not enough candles per day, skipping... NMY 34.2
Not enough candles per day, skipping... MYE 131.8
Not enough candles per day, skipping... PHX 163.4
Not enough candles per day, skipping... SENEA 36.0
Not enough candles per day, skipping... COFS 6.0
Not enough candles per day, skipping... FBNC 195.4
Not enough candles per day, skipping... IHT 58.4
Not enough candles per day, skipping... FORR 55.4
Not enough candles per day, skipping... ATR 168.6
Not enough candles per day, skipping... EXPC 42.4
Not enough candles per day, skipping... BHV 4.2
Not enough candles per day, skipping... CVCY 32.8
Not enough candles per day, skipping... BATRK 144.6
Not enough candles per day, skipping... ELOX 61.6
Not enough candles per day, skipping... SCVL 185.4
Not enough candles per day, skipping... HPI 117.8
Not enough candles per day, skipping... PF

Not enough candles per day, skipping... ATLC 35.2
Not enough candles per day, skipping... BYFC 147.6
Not enough candles per day, skipping... PMO 54.8
Not enough candles per day, skipping... HNI 169.0
Not enough candles per day, skipping... ERYP 10.8
Not enough candles per day, skipping... FARO 105.2
Not enough candles per day, skipping... RICK 109.4
Not enough candles per day, skipping... HLG 22.2
Not enough candles per day, skipping... NH 181.2
Not enough candles per day, skipping... SIF 8.0
Not enough candles per day, skipping... VEC 103.0
Not enough candles per day, skipping... SSD 161.8
Not enough candles per day, skipping... DLPN 139.2
Not enough candles per day, skipping... LKFN 105.8
Not enough candles per day, skipping... MOV 136.8

1 Failed download:
- BREW: No data found for this date range, symbol may be delisted
Error: BREW 'Datetime'
Not enough candles per day, skipping... MLND 145.2
Not enough candles per day, skipping... NCSM 22.0
Not enough candles per day, skipping... 

Incomplete data, skipping... LIVK 4
Not enough candles per day, skipping... JCE 71.2
3501 4253
Not enough candles per day, skipping... PETZ 160.2
Not enough candles per day, skipping... DXR 5.6
Incomplete data, skipping... FSEA 4
Not enough candles per day, skipping... MLVF 17.2
Not enough candles per day, skipping... RMAX 114.4
Not enough candles per day, skipping... MATW 162.4
Not enough candles per day, skipping... ARQT 182.6
Not enough candles per day, skipping... BCV 44.8
Not enough candles per day, skipping... PBPB 135.6
Not enough candles per day, skipping... JHS 42.6
Not enough candles per day, skipping... RDY 188.4

1 Failed download:
- PTLA: No data found for this date range, symbol may be delisted
Error: PTLA 'Datetime'
Not enough candles per day, skipping... FFNW 29.4
Not enough candles per day, skipping... KAI 54.6
Not enough candles per day, skipping... UG 13.8
Not enough candles per day, skipping... NML 191.2
Not enough candles per day, skipping... PWOD 11.8
Not enough c

Not enough candles per day, skipping... DCOM 109.6
Not enough candles per day, skipping... CEL 24.6

1 Failed download:
- HCCO: No data found for this date range, symbol may be delisted
Error: HCCO 'Datetime'
Not enough candles per day, skipping... WMK 125.4
Not enough candles per day, skipping... CPS 156.2
Not enough candles per day, skipping... LQDT 154.6
Not enough candles per day, skipping... PCPL 176.6
Not enough candles per day, skipping... BANF 101.6
Not enough candles per day, skipping... CHSCP 19.6
Not enough candles per day, skipping... INSE 84.4
Not enough candles per day, skipping... ALRS 23.8
Not enough candles per day, skipping... KSPN 4.2
Not enough candles per day, skipping... JMP 49.6
Not enough candles per day, skipping... HBNC 126.6
Not enough candles per day, skipping... NOVSU 21.2
Not enough candles per day, skipping... AEF 44.6
Not enough candles per day, skipping... HYMC 121.0
Not enough candles per day, skipping... DFPH 49.2
Not enough candles per day, skipping.

Not enough candles per day, skipping... PFN 144.4
Not enough candles per day, skipping... MYC 70.8
Not enough candles per day, skipping... TEI 152.6
Not enough candles per day, skipping... HMLP 153.8
Not enough candles per day, skipping... VCV 124.8
Not enough candles per day, skipping... BRLIU 5.4
Incomplete data, skipping... HYMCW 2
Not enough candles per day, skipping... AMS 61.2
Not enough candles per day, skipping... PAC 154.6
Not enough candles per day, skipping... OBCI 38.2

1 Failed download:
- FVAC: No data found for this date range, symbol may be delisted
Error: FVAC 'Datetime'

1 Failed download:
- FORK: No data found for this date range, symbol may be delisted
Error: FORK 'Datetime'
Not enough candles per day, skipping... VOLT 74.6
Not enough candles per day, skipping... PYPD 56.2
Not enough candles per day, skipping... IHIT 122.4
Not enough candles per day, skipping... OXLCP 14.4
Not enough candles per day, skipping... MPX 48.2
Not enough candles per day, skipping... CHCT 

# Update table: prices_d

In [4]:
# get max date present
q = '''
    SELECT DATE(MAX(date))
      FROM prices_d
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# check dates
end = add_days(datetime.datetime.today().strftime('%Y-%m-%d'), 3) #today's date plus 3 days
df = yf.download('IBM', start=max_date_str, end=end, interval='1d', progress=0).reset_index()
df = df[df['Date'].astype('str')>max_date_str]
assert not df.empty, 'No dates to extract...'
print(f'Extracting from {max_date_str} plus 1, to {end}')

Extracting from 2020-11-27 plus 1, to 2020-12-09


In [5]:
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract
dt_cols = {
    'sym':'sym',
    'Date':'date',
    'Open':'open',
    'High':'high',
    'Low':'low',
    'Adj Close':'adj_close',
    'Volume':'volume',
}
ls_df = []
for i, sym in enumerate(ls_sym):
    try:
        if i%300==0:
            print(i+1, len(ls_sym))
        #df = yf.download(sym, start=max_date_str, end=end, interval='1d', progress=0).reset_index()
        #df = df[df['Date'].astype('str')>max_date_str]
        df = yf.download(sym, period = '1mo', interval='1d', progress=0).reset_index()
        df['sym'] = sym
        df = df.rename(columns=dt_cols)
        df = df[list(dt_cols.values())]
        ls_df.append(df)
    except Exception as e:
        print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
if ls_df:
    # load
    df = pd.concat(ls_df)
    df.to_sql('prices_d', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_d(db, max_date_str)
    beeps()

1 4253

1 Failed download:
- GPAQ: No data found for this date range, symbol may be delisted

1 Failed download:
- FMCI: No data found for this date range, symbol may be delisted

1 Failed download:
- MJCO: No data found for this date range, symbol may be delisted

1 Failed download:
- SPAQ: No data found for this date range, symbol may be delisted

1 Failed download:
- LVGO: No data found for this date range, symbol may be delisted

1 Failed download:
- CNXM: No data found for this date range, symbol may be delisted
301 4253

1 Failed download:
- GARS: No data found for this date range, symbol may be delisted

1 Failed download:
- MINI: No data found for this date range, symbol may be delisted

1 Failed download:
- AIMT: No data found for this date range, symbol may be delisted

1 Failed download:
- PFNX: No data found for this date range, symbol may be delisted
601 4253

1 Failed download:
- JFK: No data found for this date range, symbol may be delisted

1 Failed download:
- IOTS: No

# Manual Editting

In [70]:
#execute
assert 0
q = '''
ALTER TABLE trading_days
RENAME COLUMN Date TO date
'''
q = '''
drop table trading_days
'''
q='''
ALTER TABLE prices_interday
  RENAME TO prices_d
'''
q='''
drop index index_prices_interday
'''

q='''
    UPDATE prices_m
       SET is_reg_hours = CASE
                           WHEN time(datetime) < time('09:30:00') THEN 0
                           WHEN time(datetime) > time('15:59:00') THEN 0
                           ELSE 1
                           END
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
q='''
    --UPDATE prices_m
       SET datetime = DATETIME(datetime, '-1 hours')
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
db.execute(q)
beeps()

In [8]:
#read
q = '''
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

'''
q='''
    select type, name, tbl_name, sql
    FROM sqlite_master
    WHERE type='index'
'''
q='''
    SELECT 
    --stocks.sym
           stocks.long_name
           ,stocks.sec
           ,stocks.ind
           ,stocks.quote_type
           ,stocks.fund_family
           ,stocks.summary
           ,prices_d.*
      FROM prices_d
      LEFT JOIN stocks
        ON stocks.sym = prices_d.sym
     WHERE DATE(date) = (SELECT MAX(DATE(date)) FROM prices_d WHERE sym='IBM')
       AND adj_close > 5
       AND volume > 100000
'''
df = pd.read_sql(q, db.conn)
#df.to_csv('temp.csv', index=0)
df.iloc[:]

,long_name,sec,ind,quote_type,fund_family,summary,sym,date,open,high,low,adj_close,volume
0,"CareDx, Inc",Healthcare,Diagnostics & Research,EQUITY,None,"CareDx, Inc., a precision medicine company, di...",CDNA,2020-12-04 00:00:00,61.939999,65.285004,60.020000,64.910004,467900
1,"MacroGenics, Inc.",Healthcare,Biotechnology,EQUITY,None,"MacroGenics, Inc., a biopharmaceutical company...",MGNX,2020-12-04 00:00:00,22.840000,24.190001,22.709999,24.000000,563700
2,"Advanced Energy Industries, Inc.",Industrials,Electrical Equipment & Parts,EQUITY,None,"Advanced Energy Industries, Inc. designs, manu...",AEIS,2020-12-04 00:00:00,98.080002,101.870003,98.080002,100.919998,444500
3,Calamos Convertible Opportunities and Income Fund,Financial Services,Asset Management,EQUITY,None,Calamos Convertible Opportunities and Income F...,CHI,2020-12-04 00:00:00,13.070000,13.280000,13.070000,13.250000,210300
4,"Exponent, Inc.",Industrials,Consulting Services,EQUITY,None,"Exponent, Inc., together with its subsidiaries...",EXPO,2020-12-04 00:00:00,81.949997,83.300003,81.949997,83.099998,193800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,"MiMedx Group, Inc.",Healthcare,Medical Devices,EQUITY,None,"MiMedx Group, Inc., a biopharmaceutical compan...",MDXG,2020-12-04 00:00:00,6.500000,6.600000,6.150000,6.190000,570300
2392,Bally's Corporation,Consumer Cyclical,Resorts & Casinos,EQUITY,None,Bally's Corporation owns and operates gaming a...,BALY,2020-12-04 00:00:00,45.200001,45.939999,43.520000,45.709999,360300
2393,"USA Technologies, Inc.",Technology,Software—Infrastructure,EQUITY,None,"USA Technologies, Inc. provides wireless netwo...",USAT,2020-12-04 00:00:00,8.770000,9.810000,8.770000,9.440000,291600
2394,"Wrap Technologies, Inc.",Technology,Scientific & Technical Instruments,EQUITY,None,"Wrap Technologies, Inc., a security technology...",WRAP,2020-12-04 00:00:00,5.350000,5.625000,5.350000,5.530000,336000


In [41]:
beeps()

In [5]:
get_df_prices('BYND', '2020-11-28', '2020-12-03')

,sym,datetime,open,high,low,adj_close,volume,is_reg_hours
133,BYND,2020-11-30 05:21:00,141.900000,141.900000,141.900000,141.900000,0,0
134,BYND,2020-11-30 07:07:00,140.360000,140.360000,140.350000,140.350000,0,0
135,BYND,2020-11-30 08:17:00,140.530000,140.530000,140.530000,140.530000,0,0
136,BYND,2020-11-30 08:18:00,140.900000,140.900000,140.900000,140.900000,0,0
137,BYND,2020-11-30 08:33:00,141.180000,141.250000,141.180000,141.250000,0,0
...,...,...,...,...,...,...,...,...
1066,BYND,2020-12-02 10:50:00,135.070007,135.388702,134.800003,135.070007,11643,1
1067,BYND,2020-12-02 10:51:00,134.860001,135.559998,134.800003,135.490005,14073,1
1068,BYND,2020-12-02 10:52:00,135.489899,135.930206,135.445007,135.930206,10986,1
1069,BYND,2020-12-02 10:53:00,135.759995,136.339996,135.759995,136.294998,14608,1


In [19]:
dt_info = yf.Ticker('CDNA').info